In [7]:
import os, shutil, random, glob
import cv2
import numpy as np
import pandas as pd

from keras.utils import plot_model  
from keras.applications.resnet50 import ResNet50  
from keras.applications.vgg16 import VGG16  
from keras.applications.inception_v3 import InceptionV3  
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model, load_model  
from keras.optimizers import SGD  
from keras.preprocessing.image import ImageDataGenerator  
import matplotlib.pyplot as plt  

In [8]:
resize = 224
def load_data():
    imgs = os.listdir("./train/")
    num = len(imgs)
    train_data = np.empty((5000, resize, resize, 3), dtype="int32")
    train_label = np.empty((5000, ), dtype="int32")
    test_data = np.empty((5000, resize, resize, 3), dtype="int32")
    test_label = np.empty((5000, ), dtype="int32")
    for i in range(5000):
        if i % 2:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 1
        else:
            train_data[i] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            train_label[i] = 0
    for i in range(5001, 10000):
        if i % 2:
            test_data[i-5000] = cv2.resize(cv2.imread('./train/' + 'dog.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-5000] = 1
        else:
            test_data[i-5000] = cv2.resize(cv2.imread('./train/' + 'cat.' + str(i) + '.jpg'), (resize, resize))
            test_label[i-5000] = 0
    return train_data, train_label, test_data, test_label

In [9]:
train_data, train_label, test_data, test_label = load_data()
train_data, test_data = train_data.astype('float32'), test_data.astype('float32')
train_data, test_data = train_data/255, test_data/255

In [10]:
test_label.shape

(5000,)

In [11]:
base_model = VGG16(weights='imagenet', include_top=False, pooling=None, input_shape=(resize, resize, 3), classes = 2)
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [12]:
model.fit(train_data, train_label,
          batch_size = 64,
          epochs = 50,
          validation_split = 0.2,
          shuffle = True)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 17s 4ms/step - loss: 0.5594 - acc: 0.7343 - val_loss: 0.4125 - val_acc: 0.8770
Epoch 2/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.3915 - acc: 0.8420 - val_loss: 0.3402 - val_acc: 0.8540
Epoch 3/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.3108 - acc: 0.8772 - val_loss: 0.2534 - val_acc: 0.9040
Epoch 4/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.2658 - acc: 0.8930 - val_loss: 0.2869 - val_acc: 0.8660
Epoch 5/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.2365 - acc: 0.9060 - val_loss: 0.2318 - val_acc: 0.9080
Epoch 6/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.2238 - acc: 0.9105 - val_loss: 0.2227 - val_acc: 0.9090
Epoch 7/50
4000/4000 [==============================] - 14s 4ms/step - loss: 0.2107 - acc: 0.9145 - val_loss: 0.1919 - val_acc: 0.9150
Epoch 8

In [13]:
scores = model.evaluate(train_data, train_label, verbose = 1)
print(scores)

5000/5000 [==============================] - 14s 3ms/step
[0.10275604767203331, 0.9636]


In [14]:
scores = model.evaluate(test_data, test_label, verbose = 1)
print(scores)

5000/5000 [==============================] - 14s 3ms/step
[188975.38695730382, 0.934]
